# Rumination prediction with cesium

### Imports

In [ ]:
%load_ext lab_black
import os
import pickle
from time import time
import pywt
import mne
import scipy
import scipy.stats
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import cesium.featurize
from time import sleep
from random import shuffle
from plotly.subplots import make_subplots
from ipywidgets import Dropdown, FloatRangeSlider, IntSlider, FloatSlider, interact

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor


from utils import *

In [ ]:
# ignore np.corrcoef RuntimeWarnings
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# from IPython.display import HTML

# display(HTML('<span style="color: #ff0000">red</span>'))

In [ ]:
# %%time
# grid_search = GridSearchCV(
#     pipeline,
#     regressor_params,
#     cv=4,
#     scoring={"r2": "r2"},
#     refit="r2",
#     n_jobs=-1,
#     verbose=10,
# )
# grid_search.fit(X_train, y_train)
# predictions = grid_search.predict(X_test)
# r2 = grid_search.score(X_test, y_test)

### Loading data

Loading EEG data and data from rumination questionnaire. By default create_df_data loads all info from given file but one can specify it by passing a list of desired labels from csv file.

In [ ]:
tmin, tmax = -0.1, 0.6
signal_frequency = 256
ERROR = 0
CORRECT = 1

In [ ]:
df_name = "go_nogo_df"
pickled_data_filename = "../data/" + df_name + ".pkl"
info_filename = "../data/Demographic_Questionnaires_Behavioral_Results_N=163.csv"

# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_df = pd.read_pickle(pickled_data_filename)
    print("Done")
else:
    print("Pickled file not found. Loading data...")
    epochs_df = create_df_data(info_filename=info_filename)
    epochs_df.name = df_name
    # save loaded data into a pickle file
    epochs_df.to_pickle("../data/" + epochs_df.name + ".pkl")
    print("Done. Pickle file created")

Data is now read into dataframe and each epoch is a single record.

Sorting participants by the number of errors, descending. This way the best participants are first.

In [ ]:
# add new columns with info about error/correct responses amount
grouped_df = epochs_df.groupby("id")
epochs_df["error_sum"] = grouped_df[["marker"]].transform(
    lambda x: (x.values == ERROR).sum()
)
epochs_df["correct_sum"] = grouped_df[["marker"]].transform(
    lambda x: (x.values == CORRECT).sum()
)

# mergesort for stable sorting
epochs_df = epochs_df.sort_values("error_sum", ascending=False, kind="mergesort")

## Training and predictions

- Computes ICA and then at each channel computes CWT (ica_n_components = N).
- For each band (frequency) from CWT set it computes features given in feature_dict parameter (eg. std or mean).
- Then it computes PCA on flattened ICA channels and features (outer_components = N)
- Ending feature vector has shape: outer_components from (ica_n_components * len(feature_dict) * frequencies)

Subset of standard features for EEG analysis provided by Guo et al. (2012)

### Regressions grid search

**Warning! It takes a lot of time! One run with ICA_n = 35 takes ~20 min** 

It is a pipeline which allows manipulation of vectorization's parameters. Base_steps dictionary consists of all steps of vectorization including standarization of data.

In rate_regression function, using GridSearchCV, cross-validation splitting strategy can be specified. Default cv = 5.
Results of cross-validated search are in **grid_search.cv_results** and chosen model is in **grid_search.best_estimator_**

Defined data transformers - custom data transformation steps

In [ ]:
def std_signal(t, m, e):
    return np.std(m)


def abs_diffs_signal(t, m, e):
    return np.sum(np.abs(np.diff(m)))


def peak_ind(t, m, e):
    return np.argmax(np.abs(m))


def peak_value(t, m, e):
    ind = np.argmax(np.abs(m))
    return m[ind]


shape_features = {
    "std": std_signal,
    "abs_diffs": abs_diffs_signal,
}

peak_features = {
    "peak_ind": peak_ind,
    "peak_value": peak_value,
}

In [ ]:
class IcaPreprocessing(TransformerMixin, BaseEstimator):
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        timepoints_per_channel = np.concatenate(X, axis=1)
        return timepoints_per_channel.T


class Cwt(TransformerMixin, BaseEstimator):
    def __init__(self, timepoints_count, mwt="morl", cwt_density=2):
        super().__init__()
        self.timepoints_count = timepoints_count
        self.mwt = mwt
        self.cwt_density = cwt_density

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_ica_transposed = X.T
        ica_n_components = X.shape[1]

        epochs_count = int(X_ica_transposed.shape[1] / self.timepoints_count)
        data_per_channel = X_ica_transposed.reshape(
            ica_n_components, epochs_count, self.timepoints_count
        )

        cwt_per_channel = []
        for data in data_per_channel:
            data_cwt = np.array(
                [cwt(epoch, self.mwt, self.cwt_density) for epoch in data]
            )
            cwt_per_channel.append(data_cwt)
        cwt_per_channel = np.array(cwt_per_channel)
        return cwt_per_channel


class CwtFeatureVectorizer(TransformerMixin, BaseEstimator):
    def __init__(self, feature_dict):
        super().__init__()
        self.feature_dict = feature_dict

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        vectorized_data = []
        for data_cwt in X:
            # cesium functions
            feature_set_cwt = cesium.featurize.featurize_time_series(
                times=None,
                values=data_cwt,
                errors=None,
                features_to_use=list(self.feature_dict.keys()),
                custom_functions=self.feature_dict,
            )
            features_per_epoch = feature_set_cwt.to_numpy()
            vectorized_data.append(features_per_epoch)
        vectorized_data = np.array(vectorized_data)
        return vectorized_data


# reshape data from (channels x epoch x features) to (epochs x channles x features)
# and then flatten it to (epoch x channels*features)
class PostprocessingTransformer(TransformerMixin, BaseEstimator):
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        vectorized_data = np.stack(X, axis=1)
        epochs_per_channel_feature = vectorized_data.reshape(
            vectorized_data.shape[0], -1
        )
        return epochs_per_channel_feature


class PCAForEachChannel(TransformerMixin, BaseEstimator):
    def __init__(self, n_components=3, random_state=0):
        super().__init__()
        self.n_components = n_components
        self.random_state = random_state

    def fit(self, X, y=None):
        # X has a shape CHANNELS x EPOCHS x FREQ x TIMEPOINTS
        # or CHANNELS x EPOCHS x ...
        self.PCAs = []
        for channel in X:
            flattened = channel.reshape(channel.shape[0], -1)
            # flattened has a shape EPOCHS x (FREQ*TIMEPOINTS)
            pca = PCA(n_components=self.n_components, random_state=self.random_state)
            pca.fit(flattened)
            self.PCAs.append(pca)
        return self

    def transform(self, X, copy=True):
        features = []
        for channel, pca in zip(X, self.PCAs):
            flattened = channel.reshape(channel.shape[0], -1)
            # flattened has a shape EPOCHS x (FREQ*TIMEPOINTS)
            ch_transformed = pca.transform(flattened)
            features.append(ch_transformed)

        features = np.array(features)
        # transform it from shape ICA_COMP x EPOCH x WAVELET_COMP
        #                      to EPOCH x ICA_COMP x WAVELET_COMP
        features = features.transpose((1, 0, 2))

        # flatten features into shape EPOCH x (ICA_COMP*WAVELET_COMP)
        features = features.reshape(features.shape[0], -1)
        return features

In [ ]:
X = np.array(epochs_df[epochs_df["marker"] == ERROR]["epoch"].to_list())
y = np.array(epochs_df[epochs_df["marker"] == ERROR]["Rumination Full Scale"].to_list())

In [ ]:
# fmt: off
base_steps = [
    ("ica_preprocessing", IcaPreprocessing()),
    ("ica", FastICA(random_state=0)),
    ("featurize", FeatureUnion([
        ('shape', Pipeline([
            ("cwt", Cwt(mwt="morl", timepoints_count=X.shape[-1])),
            ("cwt_feature", CwtFeatureVectorizer(feature_dict=shape_features)),
            ("pca", PCAForEachChannel(n_components=3, random_state=0)),
        ])),
        ('peaks', Pipeline([
            ("cwt", Cwt(mwt="mexh", timepoints_count=X.shape[-1])),
            ("cwt_feature", CwtFeatureVectorizer(feature_dict=peak_features)),
            ("pca", PCAForEachChannel(n_components=3, random_state=0)),
        ])),
    ])),
    ("scaler", StandardScaler()),
    ("svr", SVR()),
    # ("regression", Lasso()),
]
# fmt: on

# base_steps = [
#     ("ica_preprocessing", IcaPreprocessingTransformer()),
#     ("ica", FastICA(random_state=0)),
#     ("cwt", Cwt(mwt="morl", timepoints_count=X.shape[-1])),
#     ("cwt_feature", CwtFeatureVectorizer(feature_dict=shape_features)),
#     ("pca", PCAForEachChannel(random_state=0)),
#     ("scaler", StandardScaler()),
#     ("svr", SVR()),
#     # ("regression", Lasso()),
# ]

# base_steps = [
#     ("ica_preprocessing", IcaPreprocessingTransformer()),
#     ("ica", FastICA(random_state=0)),
#     ("cwt", Cwt(timepoints_count=X.shape[-1])),
#     ("cwt_feature", CwtFeatureVectorizer(feature_dict=guo_features)),
#     ("postprocessing", PostprocessingTransformer()),
#     ("pca", PCA(random_state=0)),
#     ("scaler", StandardScaler()),
#     ("svr", SVR()),
#     # ("regression", Lasso()),
# ]

# base_steps = [
#     ("ica_preprocessing", IcaPreprocessingTransformer()),
#     ("ica", FastICA(random_state=0)),
#     ("cwt", Cwt(timepoints_count=X.shape[-1])),
#     ("pca", PCAForEachChannel(random_state=0)),
#     ("scaler", StandardScaler()),
#     # ("svr", SVR()),
#     ("regression", Lasso()),
# ]

regressor_params = dict(
    ica__n_components=[5],
    # pca__n_components=[5],
    #     cwt__mwt=["mexh"],
    # regression__alpha=[0.3],
    # svr__C=[0.01, 0.02, 0.04, 0.1],
    svr__C=[0.1],
    # svr__epsilon=[0.1],
    # svr__kernel=["linear"],
)

In [ ]:
cachedir = "/home/filip/.erpinator_cache"
pipeline = Pipeline(base_steps, memory=cachedir)
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=0, shuffle=False
# )

# X_test, y_test = X_train, y_train

In [ ]:
%%time
print(" " * 102 + "corr           r2")

# get params randomly
all_params = list(ParameterGrid(regressor_params))
# shuffle(all_params)
    
for params in all_params:
    pipeline.set_params(**params)

    scores = []
    skf = KFold(n_splits=2)
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        corr = np.corrcoef(y_test, y_pred)[0][1]
        r2 = r2_score(y_test, y_pred)
        
        scores.append([corr, r2])
        
    # print scores
    print( f"{str(params):95}" , end=' ')
    means = np.mean(scores, axis=0)
    sems = scipy.stats.sem(scores, axis=0)
    for mean, sem in zip(means, sems):
        print(f"{mean:5.2f}±{sem:4.2f}", end="   ")
    print()